In [1]:
#cleaned up version of previous cell, using functional along with object oriented coding
# import panel as pn

import itertools

import pyvista as pv
import panel as pn
import param
import numpy as np
from PIL import ImageColor
import nibabel as nib



# from brainspace.plotting import build_plotter
from brainspace.plotting import plot_surf, build_plotter
from brainspace.datasets import load_parcellation, load_conte69
from matplotlib import interactive
import hvplot as hv
import param

from brainspace.datasets import load_conte69, load_fsa5, load_gradient, load_marker
from brainspace.plotting import plot_hemispheres




# js_files = {'jquery': 'https://code.jquery.com/jquery-1.11.1.min.js',
#             'goldenlayout': 'https://golden-layout.com/files/latest/js/goldenlayout.min.js'}
# css_files = ['https://golden-layout.com/files/latest/css/goldenlayout-base.css',
#              'https://golden-layout.com/files/latest/css/goldenlayout-dark-theme.css']

pn.extension('vtk')



In [2]:
from brainspace.mesh.mesh_io import read_surface
#Brain Display
labeling = load_parcellation('schaefer', scale=400, join=True)
surf_lh_bs, surf_rh_bs = load_conte69()
array_lh = labeling[:labeling.size // 2]
array_rh = labeling[labeling.size // 2:]
surf_lh = surf_lh_bs.VTKObject
surf_rh = surf_rh_bs.VTKObject

surf_lh_sctx_bs = read_surface("C:/Users/kowen/Downloads/dashboard/sctx_lh.gii")
surf_rh_sctx_bs = read_surface("C:/Users/kowen/Downloads/dashboard/sctx_rh.gii")
# surf_lh_sctx_bs, surf_rh_sctx_bs = load_subcortical(with_normals=False, join=False)
surf_lh_sctx = surf_lh_sctx_bs.VTKObject
surf_rh_sctx = surf_rh_sctx_bs.VTKObject

ipth = "C:/Users/kowen/Downloads/dashboard/tpl-avg_space-canonical_den-0p5mm_label-hipp_midthickness.surf.gii"
# surfsTmp = [None] * 2
# for i in range(2):
#     surfsTmp[i] = read_surface(ipth)
    # read_surface(ipth)
surf_lh_hip_bs = read_surface(ipth)
surf_lh_hip = surf_lh_hip_bs.VTKObject
surf_rh_hip_bs = read_surface(ipth)
surf_rh_hip = surf_rh_hip_bs.VTKObject

In [3]:
structs = ['Cortex', 'Subcortex', 'Hippocampus']

list_cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 
                                                              'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn', 
                                                              'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
                      'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
                      'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper', 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu',
                      'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic', 'twilight', 'twilight_shifted', 'hsv', 'Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2',
                      'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b',
                      'tab20c', 'flag', 'prism', 'ocean', 'gist_earth', 'terrain',
                      'gist_stern', 'gnuplot', 'gnuplot2', 'CMRmap',
                      'cubehelix', 'brg', 'gist_rainbow', 'rainbow', 'jet',
                      'turbo', 'nipy_spectral', 'gist_ncar']

In [4]:
# Load features
root_pth = "C:/Users/kowen/Downloads/dashboard/sub-PX070/sub-PX070/ses-01"

dict_feat = {}

# Cortical features
for h, feat in itertools.product(['lh', 'rh'], ['mz', 'adc', 'qt1', 'flair', 'thickness']):
    data = nib.load(f'{root_pth}/sub-PX070_ctx-{feat}-unthr_{h}.mgh').get_fdata().squeeze()
    dict_feat[f'ctx_{h}_{feat}'] = data
    if h == 'lh':
        surf_lh_bs.append_array(data, name=feat, at='p')
    else:
        surf_rh_bs.append_array(data, name=feat, at='p')
        
        
# # Subortical features
# for h, feat in itertools.product(['lh', 'rh'], ['mz', 'adc', 'qt1', 'flair', 'thickness']):
#     data = nib.load(f'{root_pth}/sub-PX070_sctx-{feat}-unthr_{h}.txt').get_fdata().squeeze()
#     if h == 'lh':
#         surf_lh_sctx_bs.append_array(data, name=feat, at='p')
#     else:
#         surf_rh_sctx_bs.append_array(data, name=feat, at='p')


# Hippocampal features
for h, feat in itertools.product(['lh', 'rh'], ['mz', 'adc', 'qt1', 'flair', 'thickness']):
    data = nib.load(f'{root_pth}/sub-PX070_hipp-{feat}-unthr_{h}.gii').darrays[0].data
    dict_feat[f'hip_{h}_{feat}'] = data
    if h == 'lh':
        surf_lh_hip_bs.append_array(data, name=feat, at='p')
    else:
        surf_rh_hip_bs.append_array(data, name=feat, at='p')


In [5]:

# pth_rh = '/home/oualid/Downloads/bids_examples/BIDS_MICs/derivatives/z-brains/asymmetry/sub-PX070/ses-01/sub-PX070_hipp-adc-unthr_rh.gii'
# pth_lh = '/home/oualid/Downloads/bids_examples/BIDS_MICs/derivatives/z-brains/asymmetry/sub-PX070/ses-01/sub-PX070_hipp-adc-unthr_lh.gii'
# adc_hip = np.concatenate([nib.load(pth_lh).darrays[0].data, nib.load(pth_rh).darrays[0].data])
# adc_hip.shape

In [6]:
# plot_subcortical(array_name=np.random.randn(14), ventricles=False, size=(800, 180), zoom=1.18, cmap='RdBu_r',
#               color_bar=True, color_range=(-3, 3), screenshot=False, view=['lateral', 'medial', 'medial', 'lateral'],
#               filename=None, embed_nb=True, interactive=False)

In [7]:
# plot_hemispheres(surf_lh_hip, surf_rh_hip, array_name=adc_hip, size=(800, 180), zoom=1.18, cmap='RdBu_r',
#               color_bar=True, color_range=(-3, 3), screenshot=False, view=['ventral', 'lateral', 'medial', 'dorsal'],
#               filename=None, embed_nb=True, interactive=False, )

In [8]:
view_names = ['lh_med', 'lh_lat', 'rh_med', 'rh_lat']

def add_cortical_row(surf_lh, surf_rh, array=None, struct='Cortex', cmap='RdBu', color='lightgrey', **kwargs):
    suffix = 'ctx'
    if struct == 'Subcortex':
        suffix = 'sctx'
    elif struct == 'Hippocampus':
        suffix = 'hip'

    plotters = {f'{k}_{suffix}': pv.Plotter() for k in view_names}
    panes = {}
    actors = {}
    
    array_lh = array_rh = None
    if array is not None:
        array_lh = array[:surf_lh.GetNumberOfPoints()]
        array_rh = array[surf_lh.GetNumberOfPoints():]
        
    for k, p in plotters.items():
        if k in ['lh_med_ctx', 'lh_lat_ctx', 'lh_med_sctx', 'lh_lat_sctx']:
            a = p.add_mesh(surf_lh, scalars=array_lh, cmap=cmap, color=color, **kwargs)
            # p.enable_parallel_projection()
            # p.enable_image_style()
            # p.camera_position = 'yz'
            p.view_yz()
            if k in ['lh_lat_ctx', 'lh_lat_sctx']:
                p.camera.azimuth = 180
            
        elif k in ['lh_med_hip', 'lh_lat_hip']:
            a = p.add_mesh(surf_lh, scalars=array_lh, cmap=cmap, color=color, **kwargs)
            p.view_yz()
            p.camera.azimuth = 180
            if k == 'lh_lat_hip':
                p.view_xy()
                p.camera.azimuth = 180
            
        elif k in ['rh_med_ctx', 'rh_lat_ctx', 'rh_med_sctx', 'rh_lat_sctx']:
            a = p.add_mesh(surf_rh, scalars=array_rh, cmap=cmap, color=color, **kwargs)
            # p.enable_parallel_projection()
            # p.enable_image_style()
            p.camera_position = 'yz'
            # p.view_yz()
            
            if k in ['rh_lat_ctx', 'rh_lat_sctx']:
                p.camera.azimuth = 180
                
        elif k in ['rh_med_hip', 'rh_lat_hip']:
            a = p.add_mesh(surf_lh, scalars=array_lh, cmap=cmap, color=color, **kwargs)
            
            p.view_xy()
            if k == 'rh_lat_hip':
                p.view_yz()    
                # p.camera.azimuth = 180
                
            # p.camera.tight(padding=0.0, adjust_render_window=True, view='yz')
                
        # p.camera.enable_parallel_projection()
        # p.camera.parallel_scale *= 3
        # print(k, p.camera.parallel_projection, p.camera.parallel_scale)
        # p.camera.zoom(2)
        # p.camera.tight()
        
        p.mapper.scalar_visibility = False
#         p.mapper.scalar_map_mode = 'point'
#         # p.mapper.scalar_map_mode = 'point_field'
        
#         p.mapper.color_mode = 'map'
        # p.mapper.interpolate_before_map = True
        # p.mapper.UseLookupTableScalarRangeOn()
    
        
        # panes[k] = pn.panel(p.ren_win, background="red")
        panes[k] = pn.panel(p.ren_win)
        actors[k] = a
                

    return plotters, panes, actors

In [9]:
# np.concatenate([dict_feat['ctx_lh_qt1'], dict_feat['ctx_rh_qt1']])

In [10]:
# plotters_ctx, panes_ctx, _ = add_cortical_row(surf_lh, surf_rh, array=None, struct='Cortex', show_scalar_bar=False, interpolate_before_map=False)
plotters_ctx, panes_ctx, _ = add_cortical_row(surf_lh, surf_rh, array=np.concatenate([dict_feat['ctx_lh_qt1'], dict_feat['ctx_rh_qt1']]), struct='Cortex', show_scalar_bar=False, interpolate_before_map=False)
plotters_sctx, panes_sctx, _ = add_cortical_row(surf_lh_sctx, surf_rh_sctx, array=None, struct='Subcortex', show_scalar_bar=False, interpolate_before_map=False)
plotters_hip, panes_hip, _ = add_cortical_row(surf_lh_hip, surf_rh_hip, array=None, struct='Hippocampus', show_scalar_bar=False, interpolate_before_map=False)
# plotters_hip, panes_hip, _ = add_cortical_row(surf_lh_hip, surf_rh_hip, array=np.concatenate([dict_feat['hip_lh_qt1'], dict_feat['hip_rh_qt1']]), struct='Hippocampus', show_scalar_bar=False, interpolate_before_map=False)
panes = panes_ctx | panes_sctx | panes_hip
plotters = plotters_ctx | plotters_sctx | plotters_hip

In [11]:
# pn.Row(panes['lh_lat_ctx'], panes['lh_med_ctx'], panes['rh_lat_ctx'], panes['rh_med_ctx'])

In [12]:
# pn.Row(panes['lh_lat_sctx'], panes['lh_med_sctx'], panes['rh_lat_sctx'], panes['rh_med_sctx'])

In [13]:
# pn.Row(panes['lh_lat_hip'], panes['lh_med_hip'], panes['rh_lat_hip'], panes['rh_med_hip'])

In [14]:
# m.interpolate_before_map = 'aaaa'

In [15]:
# "default", "point", "cell", "point_field", "cell_field" or "field".

In [16]:
# "default", "direct" or "map"

In [17]:
# # Creation of a mapping between Custom name and the VTK object reference
# actor_ref = ["None", actor.__this__]
# actor_names = ["None", 'St Helen']
# actor_opts = {k:v for k,v in zip(actor_names, actor_ref)}

# options = {}
# actor_selection = pn.widgets.Select(value=None, options=actor_opts, margin=0, name="Actor Selection")
# # actor_selection



struct_selection = pn.widgets.Select(value='None', options=['None', 'Cortex', 'Subcortex', 'Hippocampus'], margin=0, name="Structure")

In [18]:
bg_col = (1, 1, 1)
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in bg_col])
bind_and_orient = pn.widgets.CheckBoxGroup(
    value=['Orientation Widget', 'Key Bindings'], options=['Orientation Widget', 'Key Bindings']
)
reset_camera = pn.widgets.Button(name='Reset Camera')
background_color = pn.widgets.ColorPicker(value=color, name='Background Color')
scene_props = pn.WidgetBox(bind_and_orient, reset_camera, background_color, sizing_mode='stretch_width')

# Light properties
light_box_title = pn.widgets.StaticText(value='Light properties')
light_type = pn.widgets.Select(value='HeadLight', options=['HeadLight', 'SceneLight', 'CameraLight'])
light_intensity = pn.widgets.FloatSlider(start=0, end=1, value=1, name="Intensity")
light_props = pn.WidgetBox(light_box_title, light_type, light_intensity, sizing_mode='stretch_width')

# pn.Column(scene_props, light_props, max_width=320)

In [19]:
opacity = pn.widgets.FloatSlider(value=1, start=0, end=1, step=0.01, name='Opacity', disabled=True)
lighting = pn.widgets.Toggle(value=True, name='Lighting', disabled=True)
interpolation = pn.widgets.Select(value='Phong', options=['Flat','Phong'], name='Interpolation', disabled=True)
edges = pn.widgets.Toggle(value=False, name='Show Edges', disabled=True)
edges_color = pn.widgets.ColorPicker(value='#ffffff', name='Edges Color', disabled=True)
representation = pn.widgets.Select(value='Surface', options=['Points','Wireframe','Surface'], name='Representation', disabled=True)
frontface_culling = pn.widgets.Toggle(value=False, name='Frontface Culling', disabled=True)
backface_culling = pn.widgets.Toggle(value=False, name='Backface Culling', disabled=True)
ambient = pn.widgets.FloatSlider(value=0, start=-1, end=1, name='Ambient', disabled=True)
diffuse = pn.widgets.FloatSlider(value=1, start=0, end=2, name='Diffuse', disabled=True)
specular = pn.widgets.FloatSlider(value=0, start=0, end=10, name='Specular', disabled=True)
specular_power = pn.widgets.FloatSlider(value=100, start=0, end=100, name='Specular Power', disabled=True)

actor_props = pn.WidgetBox(
    opacity, lighting, interpolation, edges, edges_color,
    representation, frontface_culling, backface_culling,
    ambient, diffuse, specular, specular_power
)


In [20]:
def js_inject_panes(model_name='struct_selection'):
    return """
    let panes = [];
    if (""" + model_name + """.value === 'Cortex') {
        panes = [lh_med_ctx, lh_lat_ctx, rh_lat_ctx, rh_med_ctx];
    } else if (""" + model_name + """.value === 'Subcortex') {
        panes = [lh_med_sctx, lh_lat_sctx, rh_lat_sctx, rh_med_sctx];
    } else {
        panes = [lh_med_hip, lh_lat_hip, rh_lat_hip, rh_med_hip];
    }
    """

In [21]:
# opacity.jscallback(args={"target":vtkpan, "actor_selection":actor_selection, 'actor_selected2': actor_selected2} | panes, value="""
# console.log('Actor selected:', actor_selected2.value);
# //if (actor_selection.value!="None"){

# if (actor_selected.value === 'None') return;

# let panes = [lh_med, lh_lat, rh_lat, rh_med];
# if (actor_selected.value === 'Cortical') panes = [lh_med, lh_lat, rh_lat, rh_med];
# elif (actor_selected.value === 'Cortical') panes = [lh_med_sctx, lh_lat_sctx, rh_lat_sctx, rh_med_sctx];
# else panes = [lh_med_hip, lh_lat_hip, rh_lat_hip, rh_med_hip];

# for (const p of panes) {
#     const a = p.getActors()[0]
#     a.getProperty().setOpacity(source.value)
#     p.renderer_el.getRenderWindow().render()
# }

# """)

In [22]:
# opacity.jscallback(args={'struct_selection': struct_selection} | panes, value="""
# console.log('Structure:', struct_selection.value, source.value);

# if (struct.value === 'None') return;

# """ + js_inject_panes() + """

# for (const p of panes) {
#     const a = p.getActors()[0]
#     a.getProperty().setOpacity(source.value)
#     p.renderer_el.getRenderWindow().render()
# }

# """)



In [23]:
# actor_selected2.jscallback?

In [24]:
cb = opacity.jscallback(args={'struct_selection': struct_selection} | panes, value="""
console.log('Structure:', struct_selection.value, source.value);
if (struct_selection.value === 'None') return;

""" + js_inject_panes() + """

for (const p of panes) {
    const a = p.getActors()[0]
    a.getProperty().setOpacity(source.value)
    p.renderer_el.getRenderWindow().render()
}

""")


struct_selection.jscallback(args={"opacity":opacity} | panes, value="""

console.log('struct_selection cb:', source.value);

""" + js_inject_panes(model_name='source') + """




for (const p of panes) {
    if (source.value !== 'None') {
        const a = p.getActors()[0]
        
        
        console.log('mapper:', a.getMapper());
        console.log('input :', a.getMapper().getInputData());
        console.log('pd    :', a.getMapper().getInputData().getPointData());
        console.log('n     :', a.getMapper().getInputData().getPointData().getNumberOfArrays());
        console.log('scalar:', a.getMapper().getInputData().getPointData().getScalars());
        console.log('scalar:', a.getMapper().getInputData().getPointData().getArrayByIndex(0));
        console.log('scalar:', a.getMapper().getInputData().getPointData().getArrayByIndex(1));
        console.log('scalar:', a.getMapper().getInputData().getPointData().getArrayName(0));
        console.log('scalar:', a.getMapper().getInputData().getPointData().getArrayName(1));
        
        const properties = a.getProperty()
        
        //opacity.setv({value: properties.getOpacity()}, {silent: true})
        opacity.value = properties.getOpacity();
        
        // enable widgets modifications
        opacity.disabled = false
    } else {
        opacity.disabled = true
    }
    p.renderer_el.getRenderWindow().render()
}

""");


In [25]:
feat_selection = pn.widgets.Select(value='None', options=['None', 'ADC', 'MZ', 'FLAIR', 'qT1', 'Thickness', 'All'], name="Feature")
threshold = pn.widgets.FloatSlider(value=1, start=0, end=1, step=0.01, name='Threshold', disabled=True, tooltips=False)
cmap_selection = pn.widgets.Select(value='viridis', options=list_cmaps, name="Colormap")

report_props = pn.WidgetBox(feat_selection, threshold, cmap_selection)

In [26]:
# cb = feat_selection.jscallback(args={'struct_selection': struct_selection, 'martirio': [1, 2, 3, 4]} | panes | dict_feat, value="""
# //import vtkDataArray from 'vtk.js/Sources/Common/Core/DataArray';
# console.log('Structure:', struct_selection.value, source.value, martirio, ctx_lh_adc);


# const panes = [lh_med_ctx, lh_lat_ctx, rh_lat_ctx, rh_med_ctx, lh_med_sctx, lh_lat_sctx, rh_lat_sctx, rh_med_sctx, lh_med_hip, lh_lat_hip, rh_lat_hip, rh_med_hip];
# let pointData, newArray;
# for (const p of panes) {
#     const a = p.getActors()[0]
#     //a.getProperty().setOpacity(source.value)
    
    
#     if (source.value != 'None') {
#         console.log('here???')
#         pointData = a.getMapper().getInputData().getPointData();
#         newArray = vtkDataArray.newInstance({name: 'ADC', numberOfComponents: 1, values: Float32Array.from(ctx_lh_adc)}),
#         pointData.addArray(newArray)
#         pointData.setScalars(newArray)
#         console.log('getNumberOfArrays', pointData.getNumberOfArrays())
#         console.log('getScalars', pointData.getScalars())
#         console.log('getArrayByIndex:', pointData.getArrayByIndex(0));
#         console.log('getArrayByIndex:', pointData.getArrayByIndex(1));
#         console.log('getArrayName:', pointData.getArrayName(0));
#         console.log('getArrayName:', pointData.getArrayName(1));
#     } else {
#         console.log('What!!')
#     }
#     p.renderer_el.getRenderWindow().render()
# }

# """)

# feat_selection.link

In [27]:
# plotters['lh_med_ctx'].background_color = [0.5, 0.5, 0.5]
# panes['lh_med_ctx'].synchronize()

In [28]:
# panes['lh_med_ctx'].reset_camera()
# geom_pane.synchronize()

In [29]:
def map_feat(feat):
    if feat == 'ADC': return 'adc'
    if feat == 'FLAIR': return 'flair'
    if feat == 'qT1': return 'qt1'
    if feat == 'MZ': return 'mz'
    if feat == 'Thickness': return 'thickness'
    return feat

In [30]:
def update(event):
    #layout[1].object = autompg.hvplot.scatter(x.value, y.value, c=color.value, padding=0.1)
    feat_name = map_feat(feat_selection.value)
    # print('New feat_selection:', feat_selection.value, feat_name)
    
    
    for k, p in plotters.items():
        pdata = p.mesh.GetPointData()    

        if feat_name == 'None': 
            p.mapper.ScalarVisibilityOff()
            p.mapper.UseLookupTableScalarRangeOff()
            
        elif pdata.HasArray(feat_name): 
            # print('Has Array')
            p.mapper.scalar_visibility = True
            pdata.SetActiveScalars(feat_name)
            # p.mapper.array_name = feat_name
            # p.mapper.SetArrayName(feat_name)
            # p.mapper.ScalarVisibilityOn()
            p.mapper.Modified()
            p.mapper.UseLookupTableScalarRangeOn()
            
        panes[k].synchronize()
        
        
feat_selection.param.watch(update, 'value')



def update_cmap(event):
    for k, p in plotters.items():
        p.mapper.lookup_table.apply_cmap(cmap_selection.value, 256)
        panes[k].synchronize()
        
cmap_selection.param.watch(update_cmap, 'value')

Watcher(inst=Select(name='Colormap', options=['viridis', 'plasma', ...], value='viridis'), cls=<class 'panel.widgets.select.Select'>, fn=<function update_cmap at 0x000002504141D9E0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [31]:
# plotters['lh_med_ctx'].mesh.GetPointData().SetActiveScalars?

In [32]:
# # plotters['lh_med_ctx'].mapper.cmap = 'magma' 
# plotters['lh_med_ctx'].mapper.lookup_table.apply_cmap('magma', 256)
# panes['lh_med_ctx'].synchronize()

In [33]:
# plotters['lh_med_ctx'].mapper.scalar_visibility = False
# panes['lh_med_ctx'].synchronize()

In [34]:
# plotters['lh_med_ctx'].mapper.array_name = None
# panes['lh_med_ctx'].synchronize()

In [35]:
m = plotters['lh_med_ctx'].mapper
m

DataSetMapper (0x2504077dba0)
  Scalar visibility:           False
  Scalar range:                (-12.47425651550293, 0.03320300579071045)
  Interpolate before mapping:  False
  Scalar map mode:             point
  Color mode:                  map

Attached dataset:
PolyData (0x2503ea15d80)
  N Cells:    64980
  N Points:   32492
  N Strips:   0
  X Bounds:   -6.462e+01, 4.537e-01
  Y Bounds:   -1.028e+02, 6.740e+01
  Z Bounds:   -4.470e+01, 7.548e+01
  N Arrays:   7

In [36]:
# m.GetScalarModeAsString(), m.scalar_map_mode = 'point'

In [37]:
# float_slider = pn.widgets.FloatSlider(name='Opacity', start=0, end=1, step=0.01, value=1, disabled=False, value_throttled=1, tooltips=False)

In [38]:
row_ctx = pn.Row(panes['lh_lat_ctx'], panes['lh_med_ctx'], panes['rh_lat_ctx'], panes['rh_med_ctx'])
row_sctx = pn.Row(panes['lh_lat_sctx'], panes['lh_med_sctx'], panes['rh_lat_sctx'], panes['rh_med_sctx'])
row_hip = pn.Row(panes['lh_lat_hip'], panes['lh_med_hip'], panes['rh_lat_hip'], panes['rh_med_hip'])

In [39]:
# pn.Column(
#     # "This example demonstrates the use of **VTK and pyvista** to display a *scene*",
#     pn.Row(
#         # vtkpan.servable(title='VTK - Mt. St Helens'),
#         pn.Column(
#             row_ctx,
#             row_sctx,
#             row_hip,
#         ).servable(title='VTK - Mt. St Helens'),
#         pn.Column(
#             # actor_selection,
#             # actor_selected2,
#             struct_selection,
#             pn.Tabs(
#                 ('Report', report_props),
#                 ('Scene controller', pn.Column(scene_props, light_props)),
#                 ('Actor properties',actor_props)
#             )
#         ).servable(target='sidebar')
#     ), min_height=600
# )


layout  = pn.Column(
    # "This example demonstrates the use of **VTK and pyvista** to display a *scene*",
    pn.Row(
        # vtkpan.servable(title='VTK - Mt. St Helens'),
        pn.Column(
            # actor_selection,
            # actor_selected2,
            # struct_selection,
            pn.Tabs(
                ('Report', report_props),
                # ('Scene controller', pn.Column(scene_props, light_props)),
                ('Appearance',pn.Column(struct_selection, actor_props))
            )
        ).servable(target='sidebar'),
        pn.Column(
            pn.Column(pn.pane.Markdown('#Cortex', align="center"), row_ctx),
            pn.Column(pn.pane.Markdown('#Subcortex', align="center"), row_sctx),
            pn.Column(pn.pane.Markdown('#Hippocampus', align="center"), row_hip),
            # pn.Column('#Hippocampus', row_hip),
            # row_hip,
        ).servable(title='VTK - Mt. St Helens'),
    ), min_height=600
)

layout.show()

Launching server at http://localhost:54783
